# Audit the Data

To start, I'll import the necessary modules and parse through the data, checking for errors that need to be corrected.

In [1]:
# initial imports

import xml.etree.ElementTree as ET
from collections import defaultdict
import unicodecsv
import csv
import codecs
import re
import pprint
import xml.etree.cElementTree as ET

In [2]:
# Count_tags code from the Iterative Parsing lesson of osm case study and in file 'iterative-parsing.py'.
# This counts the amount of tags of each type.

def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename, events=('start', )):
        if elem.tag not in tags:
            tags[elem.tag] = 1
        else:
            tags[elem.tag] += 1
    return tags

def test():

    tags = count_tags('sample.osm')
    pprint.pprint(tags)
    

if __name__ == "__main__":
    test()

{'member': 707,
 'nd': 50170,
 'node': 43852,
 'osm': 1,
 'relation': 43,
 'tag': 32892,
 'way': 5733}


In [3]:
# code taken from the 'tag types' lesson in the OSM case study, also in file "tag types.py"
# This code retrieves the possible errors based on the error mapping below, so that we know what to expect.

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        if lower.match(element.attrib['k']):
            keys["lower"] += 1
        elif lower_colon.search(element.attrib['k']):
            keys["lower_colon"] += 1
        elif problemchars.search(element.attrib['k']):
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys



def test():
    keys = process_map('sample.osm')
    pprint.pprint(keys)
    
if __name__ == "__main__":
    test()

{'lower': 16640, 'lower_colon': 16019, 'other': 233, 'problemchars': 0}


Now that we have the tag types and what to expect with possible errors, lets look at the users. The code below will tell us how many unique contributing users we have in the sample.

In [4]:
# The code below comes from the Exploring Users lesson in the OSM case study, and in file '7-explore-users.py'
# This code will parse out how many total unique users have added to the map data. 
# It adds each unique user to the dictionary and then returns the length of the dictionary.

def get_user(element):
    uid = ''
    if element.tag == "node" or element.tag == "way" or element.tag == "relation":
        uid = element.get('uid')
    return uid

def proc_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if get_user(element):
            users.add(get_user(element))
            users.discard('')
        pass
    return users

def test():
    users = proc_map('sample.osm')
    pprint.pprint(len(users))

if __name__ == "__main__":
    test()

351


There is invariably going to be incorrect entries into the database, and one major one may be incorrectly named street names. I'll clean those up in the sample below to better sort the data into the DB. First I'll idenitfy them, and then I'll apply the mapping to update the names into a cleaner format. 

In [5]:
%%capture

# using %%capture to reduce unwanted output from the long cells
# This code is also saved in the 'improve-street-names1.py' file.
# This code audits the street names so that we can see what might need to be fixed.

OSMFILE = "sample.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road",
            "Trail", "Parkway", "Plaza", "Park"]

#this pulls the street data from the raw string.

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


#organizes the dictionary of values

def print_sorted_dic(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print("%s: %d" % (k, v))

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

#  audits the data by iterating through the dictionary of street types and returnng values that match the tag structure

def audit():
# osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(OSMFILE, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    pprint.pprint(dict(street_types))

if __name__ == '__main__':
    audit()

In [6]:
# This code is also saved in the 'improve-street-names2.py' file.

mapping = {"St": "Street",
           "ST": "Street",
           "St.": "Street",
           "St,": "Street",
           "Street.": "Street",
           "street": "Street",
           "Sq": "Square",
           "Rd.": "Road",
           "Rd": "Road",
           "Ave": "Avenue",
           "DR.": "Drive"
           }

# audits the code again to check it against the expected values.

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osmfile, events=("start",)):

        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

# updates the errors against the mapping to replace the name with the correct form.

def update_name(name, mapping):
    for key, value in mapping.items():
        if re.search(key, name):
            name = re.sub(street_type_re, value, name)

    return name


def test():
    st_types = audit(OSMFILE)
    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print (name, "=>", better_name)


if __name__ == '__main__':
    test()

Southwest Broadway => Southwest Broadway
Northeast Broadway => Northeast Broadway
Southwest Burns Way => Southwest Burns Way
Southeast Wyndham Way => Southeast Wyndham Way
Southeast Rust Way => Southeast Rust Way
Southwest Legacy Oak Way => Southwest Legacy Oak Way
Woodglen Way => Woodglen Way
Southwest 18th Way => Southwest 18th Way
Southwest Jessica Way => Southwest Jessica Way
Southwest Kalyca Way => Southwest Kalyca Way
Southeast Sieben Park Way => Southeast Sieben Park Way
Northwest Tam O' Shanter Way => Northwest Tam O' Shanter Way
Northeast Fairview Lake Way => Northeast Fairview Lake Way
South Big Sky Way => South Big Sky Way
Southeast Deer Creek Way => Southeast Deer Creek Way
Southwest Alpine Crest Way => Southwest Alpine Crest Way
Southwest Sonnet Way => Southwest Sonnet Way
North Burgard Way => North Burgard Way
Orchard Way => Orchard Way
Southeast Megan Way => Southeast Megan Way
Southeast Vivian Way => Southeast Vivian Way
Northeast Vista Way => Northeast Vista Way
Southe

The main element fixes are the tags for Streets and the State, where incorrect data might be placed that would remove large aspects of the city from my analysis. Those will be the focus before exporting. While some streets could be wholly incorrect, in this clean-up we'll just be making the names uniform for their tag.

In [7]:
#located in 'is-state.py'
# Audits the state element to see if it shows up as 'OR' and if its doesn't, then replaces it with 'OR'.

def is_state(elem):
    return (elem.attrib['k'] == "addr:state")

state_types = defaultdict(int)

def audit_state(state_types, state_name):
    if state_name != 'OR':
        state_types[state_name] += 1

for event, elem in ET.iterparse(OSMFILE, events =("start",)):
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
            if is_state(tag):
                audit_state(state_types, tag.attrib['v'])

Just a few mislabeled state elements as WA instead of OR. There is likely many more due to the sheer size of the original extract. Far too many to fix here. But these can be replaced.

## Reshaping elements and exporting to CSV

With the street and state element functions set, now it's time to run the process_map function in the main code, and parse out the csv files I need for the database. 

In [8]:
# contained in file 'schema.py' as well as lesson 11 'Case Study Open Street Map data'

schema = {
    'node': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
            }
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
            }
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

In [9]:
#original code in the 'data.py' file also in the lesson 11 of 'Case Study OpenStreetMap data'
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET
from collections import defaultdict
import unicodecsv
import csv
import codecs
import re
import pprint
import re
import xml.etree.cElementTree as ET


# import cerberus
# Cerberus commented out because Anaconda environment refuses to accept cerberus package. 
# All validator code commented out because of this environment issue. 
# The rest of the code can still function without cerberus validator.

# import schema

OSM_PATH = "sample.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

print ("Street Names and State Codes Fixed:")
print ("\n")

# Fix Street & State Names 

#located in 'fix-street-names.py'

OSMFILE = "sample.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road",
            "Trail", "Parkway", "Plaza", "Park"]

mapping = {"St": "Street",
           "ST": "Street",
           "St.": "Street",
           "St,": "Street",
           "Street.": "Street",
           "street": "Street",
           "Sq": "Square",
           "Rd.": "Road",
           "Rd": "Road",
           "Ave": "Avenue",
           "DR.": "Drive",
           "Blvd": "Boulevard"
           }

# Auditing the street names and creating the dictionary for teh fixing element to iterate through.

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osmfile, events=("start",)):

        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

# updates the errors against the mapping to replace the name with the correct form.

def update_name(name, mapping):
    for key, value in mapping.items():
        if re.search(key, name):
            name = re.sub(street_type_re, value, name)

    return name


def test():
    st_types = audit(OSMFILE)
    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print (name, "=>", better_name)

            
# Auditing state functions to parse through for the fixing function

def is_state(elem):
    return (elem.attrib['k'] == "addr:state")

state_types = defaultdict(int)

def audit_state(state_types, state_name):
    if state_name != 'OR':
        state_types[state_name] += 1

for event, elem in ET.iterparse(OSMFILE, events =("start",)):
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
            if is_state(tag):
                audit_state(state_types, tag.attrib['v'])


# fixes the street names by checking if its in the mapping and then replacing the errors to be uniform accrodingly

def fix_street(elem):

    street_types = defaultdict(set)
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
            if is_street_name(tag):
                audit_street_type(street_types, tag.attrib['v'])

            for st_type, ways in street_types.items():
                for name in ways:
                    for key,value in mapping.items():
                        n = street_type_re.search(name)
                        if n:
                            street_type = n.group()
                            if street_type not in expected:
                                if street_type in mapping:
                                    better_name = name.replace(key,value)
                                    if better_name != name:
                                        print ("Fixed Street:", tag.attrib['v'], "=>", better_name)
                                        tag.attrib['v'] = better_name
                                        return
#is located in 'fix-state.py'
#This will replace the incorrect state (WA) with (OR).
                                    
def fix_state(elem):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_state(tag):
                    if tag.attrib['v'] != 'OR':
                        print ("Fixed State: ", tag.attrib['v'], "=> 'OR'")
                        tag.attrib['v'] = 'OR'
                                    
def fix_element(elem):
    fix_street(elem)
    fix_state(elem)
                                

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # Fix data issues, based on auditing results
    # runs the fix_element function which calls the fix state and fix street functions. 
    # This will replace the necessary errors.
    # The rest of shape_element will then run through the nodes and ways tags to make sure that all
    # of the elements are correct type for each row location.
    
    fix_element(element)

    if element.tag == 'node':

            for node_field in node_attr_fields:
                node_attribs[node_field] =element.attrib[node_field]

            for tag in element.iter('tag'):
                k = tag.attrib['k']

                # ignores tags containing problem characters in the k tag attribute:

                if re.search(PROBLEMCHARS,k):
                    continue
                else:
                    pass

                tag_dict = {}

                tag_dict['id'] = node_attribs['id']

                colon_find = re.split('[:]', k)

                if len(colon_find) == 1:

                    tag_dict['key'] = k
                    tag_dict['type'] = 'regular'

                elif len(colon_find) == 2:

                    tag_dict['key'] = colon_find[1]
                    tag_dict['type'] = colon_find[0]

                elif len(colon_find) > 2:

                    tag_dict['key'] = ':'.join(colon_find[1:])
                    tag_dict['type'] = colon_find[0]

                tag_dict['value'] = tag.attrib['v']

                tags.append(tag_dict)

            return {'node': node_attribs, 'node_tags': tags}

    elif element.tag == 'way':

        for way_field in way_attr_fields:
            way_attribs[way_field] =element.attrib[way_field]

        for tag in element.iter('tag'):
            k = tag.attrib['k']

            # ignores tags containing problem characters in the k tag attribute:

            if re.search(PROBLEMCHARS,k):
                print ("Problem character found - skipping element")
                continue
            else:
                pass

            tag_dict = {}

            tag_dict['id'] = way_attribs['id']

            colon_find = re.split('[:]', k)

            if len(colon_find) == 1:

                tag_dict['key'] = k
                tag_dict['type'] = 'regular'

            elif len(colon_find) == 2:

                tag_dict['key'] = colon_find[1]
                tag_dict['type'] = colon_find[0]

            elif len(colon_find) > 2:

                tag_dict['key'] = ':'.join(colon_find[1:])
                tag_dict['type'] = colon_find[0]

            tag_dict['value'] = tag.attrib['v']

            tags.append(tag_dict)

        n = 0
        for nd in element.iter('nd'):

            nd_dict = {}

            nd_dict['id'] = way_attribs['id']
            nd_dict['node_id'] = nd.attrib['ref']
            nd_dict['position'] = n
            way_nodes.append(nd_dict)
            n+=1

        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


        
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

# Validator elements commented out because cerberus package can't be installed. See above comments.

#def validate_element(element, validator, schema=schema):
#    """Raise ValidationError if element does not match schema"""
#    if validator.validate(element, schema) is not True:
#        field, errors = next(validator.errors.items())
#        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
#        error_string = pprint.pformat(errors)
#        
#        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow(
            {
                k: (v.encode("utf-8") if isinstance(v, str) else v)
                for k, v in row.items()
            }
        )

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w', 'utf-8') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w', 'utf-8') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w', 'utf-8') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w', 'utf-8') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w', 'utf-8') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

    # Validator elements commented out because cerberus package can't be installed. See above comments.
        # validator = cerberus.Validator()
    
    # process_map() relies on shape_element() to iterate thru the code and parse which dict the data is parsed to
    # and then it writes it to the appropriate CSV file.

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
              #  if validate is True:
               #     validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)
    
print("Reshaping and export complete.")

Street Names and State Codes Fixed:


Fixed State:  WA => 'OR'
Fixed State:  WA => 'OR'
Fixed Street: NE 22nd Ave => NE 22nd Avenue
Fixed State:  WA => 'OR'
Fixed State:  WA => 'OR'
Fixed State:  WA => 'OR'
Reshaping and export complete.


Now I need to import the database and get it set up successfully.

Since I hade some module issues with getting Anaconda to accept certain packages, I've taken the CSV files written and imported them to Microsoft Access to perform the queries there. I'll provide a graphic representation below of each of the queries.

### Size of the files

| File      | Size |
| ----------- | ----------- |
| Portland_oregonosm.xml      | 1.46 gb       |
| Sample.osm   |   10.2 mb      |
| Portland.accdb  |  14 mb  |
| nodes.csv  |  3.9 mb  |
| nodes_tags.csv  |  65 kb  |
| ways.csv   |  388 kb  |
| ways_tags.csv  |  1 mb  |
| ways_nodes.csv  |  1.1 mb  |


### The number of nodes

>SELECT COUNT(*) FROM nodes;

43852

### The number of ways

>SELECT COUNT(*) FROM ways;

5733

### Unique Users

It may be noticeable that the SQL is not regular sqlite SQL but the syntax necessary for Microsoft Access.

>SELECT COUNT(*)  
>FROM (SELECT DISTINCT uid FROM nodes UNION ALL SELECT DISTINCT uid FROM ways) e;

461

### Top 10 users

SELECT TOP 10 user, COUNT(\*) FROM  
(SELECT user FROM nodes UNION ALL SELECT user FROM ways)  
GROUP BY user  
ORDER BY COUNT(\*) DESC;  

| user	| Count  |
| ----------- | ----------- |
|  Peter Dobratz_pdxbuildings  |  13033  |
|  lyzidiamond_imports  |  12689  |
|  Mele Sax-Barnett  |  3843 |
| baradam  |  	3343  |
| Darrell_pdxbuildings  | 	2863  |
| cowdog  |  	2182  |
| Peter Dobratz  |  	2008  |
| Grant Humphries  |  	1937  | 
| justin_pdxbuildings  |  	776  |
| amillar-osm-import  |  	725  |

### Amenity with most entries

SELECT TOP 1 value, COUNT(\*)  
FROM nodes_tags  
WHERE key="amenity"  
GROUP BY value  
ORDER BY COUNT(\*) DESC;  

|  Value  |  Count  |
| -------- | ------- | 
|  bicycle_parking  |  13   |

Let's do a silly one. This is the Pacific Northwest, so lets see how many Starbucks there are in the entries.

### Number of Starbucks

SELECT count (*)  
FROM nodes_tags  
WHERE value ='STARBUCKS';  

1

## Problems in the Data and Suggestions

Auditing the sample size of Portland map, a few problems came up that could be fixed and cleaned early on, before adding to the database. Abbreviated street names and incorrect state listings were the most common.
This was fixed programmatically above using a few functions.

Now that the data is in the database, it is clear there are a few other problems in the data, which interferes with making deeper query assertations. Across the three 'nodes' tables, there are a significant amount of missing 'id' entries, which shrinks the amount of outputs for joined queries considerably. This could bring the numbers/COUNT outputs into question, as some rows may get excepted when querying nodes JOIN nodes_tags queries.

One of the key problems with datasets this large, even for just a relatively small sample of a single city, is the amount of unique points being logged. A unique id for each node tag quickly scales into the billions. Rather than an overall unique id for each tag, it would be helpful to subset the id for tags to each user and type, reducing the overall numeric scale for each tag. Even would simply require more specificty in querying as the unique user would need to be a query element as well as the id's, to differentiate if any numbers duplicated (which they would).

### Additional exploration

Below is an extra query to explore some aspects of the map, like the starbucks exploration above.

Number of religious locations:

SELECT value, COUNT(\*)  
FROM nodes_tags WHERE key="religion"  
GROUP BY value  
ORDER BY COUNT(\*) DESC;  

|  Value  |  Count  |
| ------- | ---------- |
| christian  |  2 |


# Conclusion

It's apparent even from my sample that the Portland extract of data is positively massive, and it would have been much easier to parse through with a smaller city scale. 
The most distorting factor in the data appears to be the inclusion of quite a bit of Vancouver, WA data as well, with streets and places holding WA tags. 
The data is clearly bot-submitted on many users accounts, with tag designations by the highest contributors scaling into the billions range. It would be helpful to make these designations more specific for each element, which would allow for more clean parsing when identifying problems. 

Many of these problems and solutions are beyond my ability to tackle, but I think it makes sense to try to fix them from an engineering perspective, to have cleaner data going into the map initially. This way there is less cleaning necessary on the back end.

# References

Street names code confirmation:
https://github.com/belgarion42/c750/blob/master/Udacity%20OSM%20Project%20-%20Irvine%2C%20CA%20-%20Randy%20Crane.ipynb
https://github.com/wblakecannon/udacity-dand/blob/master/4-Data-Wrangling/L12-Case-Study-OpenStreetMap-Data/11-Quiz-Preparing-for-Database-SQL.py

